In [2]:
import sqlite3
import pandas as pd

# Connect to in-memory database (or use a file with "sqlite:///mydb.sqlite")
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# Create tables
cur.execute("CREATE TABLE customers (id INTEGER PRIMARY KEY, name TEXT)")
cur.execute("CREATE TABLE orders (id INTEGER PRIMARY KEY, customer_id INTEGER, amount INTEGER)")

# Insert data
cur.executemany("INSERT INTO customers VALUES (?, ?)", [
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Clara'),
])

cur.executemany("INSERT INTO orders VALUES (?, ?, ?)", [
    (1, 1, 200),
    (2, 1, 150),
    (3, 2, 300),
    (4, 3, 100),
    (5, 2, 50),
])

conn.commit()

# Run SQL
query = """
SELECT c.name, SUM(o.amount) AS total_spent
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name
"""
df = pd.read_sql_query(query, conn)
print(df)


    name  total_spent
0  Alice          350
1    Bob          350
2  Clara          100


In [9]:
#nly the customer(s) with the maximum total_spent.
query = """
select c.name, SUM(o.amount) as total_spent
from customers c
join orders o on c.id = o.customer_id
group by c.name
having total_spent > 300
"""

In [11]:
query = """
SELECT c.name, SUM(o.amount) AS total_spent
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name
HAVING SUM(o.amount) = (
    SELECT MAX(total)
    FROM (
        SELECT SUM(o2.amount) AS total
        FROM customers c2
        JOIN orders o2 ON c2.id = o2.customer_id
        GROUP BY c2.name
    )
)
"""
df = pd.read_sql_query(query, conn)
print(df)


    name  total_spent
0  Alice          350
1    Bob          350
